In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Lee el archivo de Excel con dos columnas (texto y etiqueta)

df = pd.read_excel('cluster.xlsx')

# Separa las columnas de datos y etiquetas
df['Data'] = df['Data'].astype(str)
datos = df['Data'].tolist()
etiquetas = df['Etiqueta'].tolist()

# Preprocesamiento de datos
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(datos)
sequences = tokenizer.texts_to_sequences(datos)
word_index = tokenizer.word_index
maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)


# Codifica las etiquetas de texto a números
le = LabelEncoder()
labels = le.fit_transform(etiquetas)
labels = np.asarray(labels)

# Separa los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

# Crea el modelo
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(60, activation='softmax'))

# Compila el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convierte las etiquetas a codificación one-hot
y_train = np.eye(60)[y_train]
y_test = np.eye(60)[y_test]

# Entrena el modelo
model.fit(X_train, y_train, epochs=15, batch_size=64, validation_split=0.2)

# Evalúa el modelo en el conjunto de prueba
score, acc = model.evaluate(X_test, y_test, batch_size=64)
print("Precisión del modelo: {:.2f}%".format(acc * 100))

Epoch 1/15
